In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import os

import sys
sys.path += ['utils/']  

from EDRep import *
from dcsbm import *
from node_embedding import *

import warnings
warnings.filterwarnings("ignore")

## Test against spectral clustering

In [2]:
# set the parameters

n = 10000 # number of nodes
n_clusters = 4 # number of communities
c = 10 # average degree
c_out = 4 # across-classes connectivity
γ = 6 # degree distribution heterogeneity parameter
dim = 32 # embedding dimension

# degree distribution latent parameter
θ = np.random.uniform(3,12, n)**γ
θ = θ/np.mean(θ)
sq = np.mean(θ**2)
print(f'α = {(c-c_out)*np.sqrt(sq/c)}')

# label vector
π = np.ones(n_clusters)/n_clusters
label = np.concatenate([[j for i in range(int(π[j]*n))] for j in range(n_clusters)])

# create the C matrix
C = matrix_C(c_out, c, 0., π)

# generate the adjacency matrix
A, ℓ = adj(C,c, label, θ, True)

NMIv, tv = [], []


# Node embedding with EDRep
t0 = time()
res = NodeEmbedding(A, dim, n_epochs = 50, walk_length = 3, k = 1, verbose = True, η = 1., sym = True)
NMIv.append(computeScore(res.X, ℓ))
tv.append(time()-t0)

# Node2Vec
t0 = time()
Y = Node2VecNS(A, dim, verbose = True)
tv.append(time() - t0)
NMIv.append(computeScore(Y, ℓ))

# Spectral clustering
t0 = time()
X = community_detection(A , n_clusters = n_clusters, verbose = True)
tv.append(time()-t0)
NMIv.append(computeScore(X, ℓ))

α = 3.1901579996130858
Running the optimization for k = 1
Preprocessing progress: Learning embeddings using the Skip-Gram model...
Preprocessing progress: 99.01%Preprocessing progress: 84.66%
Preprocessing transition probabilities: done!
Walking progress: 86.1%%ing progress: %0%%0%0% progress: 00%Walking progress: 
Simulating random walks: done!
Learning embeddings: done!


Estimating zeta : 02

In [3]:
names = ['EDRep', 'Node2Vec', 'SC']

for i in range(3):
    print(f'{names[i]}\nNMI: {NMIv[i]}\nExecution time: {tv[i]}\n...................\n')

EDRep
NMI: 0.6713893947279171
Execution time: 10.220693349838257
...................

Node2Vec
NMI: 0.6501960958367452
Execution time: 16.03024125099182
...................

SC
NMI: 0.6678669637871774
Execution time: 3.2949910163879395
...................

